In [16]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [17]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [18]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [19]:
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [20]:
from expiries import dict_expiries

In [21]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [22]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{variation}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [23]:
bnf_1min = pd.read_csv("../data/nifty_min (2).csv")
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
# bnf_1min.tail()
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2019) & (bnf_1min["datetime"].dt.year <= 2025)
]

In [24]:
bnf_1min.tail()

,index,datetime,o,h,l,c,v
777613,nifty,2025-05-30 15:25:00,24741.4000,24742.7000,24740.5000,24741.7000,0
777614,nifty,2025-05-30 15:26:00,24742.2500,24746.2000,24740.3000,24740.3000,0
777615,nifty,2025-05-30 15:27:00,24741.0500,24749.0500,24739.5000,24747.1500,0
777616,nifty,2025-05-30 15:28:00,24746.5500,24746.8000,24731.1000,24745.2500,0
777617,nifty,2025-05-30 15:29:00,24743.6000,24749.3000,24731.8500,24736.6500,0


In [25]:
days_traded = bnf_1min['datetime'].dt.date.unique().tolist()

In [26]:
from datetime import date

def get_monthly_expiry(input_date: date, index_symbol: str) -> date | None:
    """
    Returns the farthest expiry in the same month and year as input_date
    for the given index_symbol using dict_expiries.
    """
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None

    # Filter expiries for the same month and year as input_date
    same_month_expiries = [
        dt.date() for dt in expiries
        if dt.year == input_date.year and dt.month == input_date.month
    ]

    return max(same_month_expiries) if same_month_expiries else None


In [27]:
index_ = 'nifty'
tf_ = '1m'
offset_ = '0m'

if index_ == 'nifty':
    LOT_SIZE_ = 75
    STRIKE_SPREAD_ = 50
    INDEX_LEVERAGE_ = 8
    PORTFOLIO_ = 1_00_00_000

In [34]:
async def send_trade(strike, opt_type, expiry, dte, current_date, current_time, entry_price, qty, tradebook):
    trade = {
        'entry date': current_date,
        'entry time': current_time,
        'index': index_,
        'strike': strike,
        'expiry': expiry,
        'dte': dte,
        'option type': opt_type,
        'entry price': entry_price,
        'qty': qty,
    }
    tradebook.append(trade)
    # print(trade)
    return tradebook

async def take_exit(strike, opt_type, exit_date, exit_time, tradebook, exit_price):
    for trade in tradebook:
        if (
            trade['option type'] == opt_type and
            trade['strike'] == strike and
            'exit time' not in trade
        ):
            trade['exit date'] = exit_date
            trade['exit time'] = exit_time
            trade['exit price'] = exit_price
            
    return tradebook

In [45]:
async def execute(df, tf, offset):
    df['datetime'] = pd.to_datetime(df['datetime'])
    start_date = dt.date(2019, 1, 1)
    end_date = dt.date(2025, 5, 31)

    tradebook = []
    search_time = dt.time(10, 0)

    current_date = start_date

    take_new_trade = False
    expiry_found = False
    rebalance = False
    current_timestamp = dt.datetime.combine(current_date, search_time)
    position_active = False
    first_trade_of_month = False
    dte = -1

    while current_date <= end_date:
        if current_date in days_traded:
            
            if not position_active and not first_trade_of_month:
                search_time = dt.time(10, 0)
                take_new_trade = True
                current_timestamp = dt.datetime.combine(current_date, search_time)
                first_trade_of_month = True
                
            spot_df = df[(df['datetime'] >= current_timestamp) & (df['datetime'].dt.date == current_date)]
            spot_df = spot_df.reset_index()
            print(current_date, current_timestamp)
            # print(spot_df.to_string())
            
            if not len(spot_df) > 1:
                current_date += dt.timedelta(days=1)
                continue
            if current_date.day > 15 and not position_active:
                current_date += dt.timedelta(days=1)
                continue
            if not expiry_found:
                spot_atm = spot_df['o'].iloc[0]
                # print(spot_atm)
                expiry = get_monthly_expiry(current_date, index_)
                dte = (expiry - current_date).days
                if dte >= 0 and len(spot_df) > 1:
                    # print(len(spot_df))
                    # print(spot_df.to_string())
                    pct_away = max(1, (dte / 10))
                    ce_strike = int(round(spot_atm * ((100 + pct_away) / 100) / STRIKE_SPREAD_) * STRIKE_SPREAD_)
                    pe_strike = int(round(spot_atm * ((100 - pct_away) / 100) / STRIKE_SPREAD_) * STRIKE_SPREAD_)
                    expiry_found = True
                    print(f'Nearest Expiry : {expiry}, Spot ATM : {spot_atm}, DTE : {dte}')
                    print(f'CE Strike : {ce_strike}, PE Strike : {pe_strike}')
                    rebalance = False
                else:
                    current_date += dt.timedelta(days=1)
                    continue
        
    
            if take_new_trade and dte >= 0 and current_date.day <= 15:
    
                ce_df = await fetch_data(
                    index=index_,
                    expiry=expiry,
                    strike=ce_strike,
                    asset_class='C',
                    start_date=current_date,
                    start_time=search_time,
                    end_date=current_date,
                    end_time=dt.time(15, 30),
                )
                if ce_df is not None and not isinstance(ce_df, str):
                    ce_df = resample(ce_df, tf, offset)
                    ce_df_pandas = ce_df.to_pandas()
                    # print(ce_df_pandas.iloc[0])
                    entry_price_ce = ce_df_pandas.iloc[0]['o']
                    qty_ce = int(round((PORTFOLIO_ * INDEX_LEVERAGE_ / ce_strike) / LOT_SIZE_) * LOT_SIZE_)
                else:
                    entry_price_ce = float('nan')
                    qty_ce = 0
    
                pe_df = await fetch_data(
                    index=index_,
                    expiry=expiry,
                    strike=pe_strike,
                    asset_class='P',
                    start_date=current_date,
                    start_time=search_time,
                    end_date=current_date,
                    end_time=dt.time(15, 30),
                )
                if pe_df is not None and not isinstance(pe_df, str):
                    pe_df = resample(pe_df, tf, offset)
                    pe_df_pandas = pe_df.to_pandas()
                    # print(ce_df_pandas.iloc[0])
                    entry_price_pe = pe_df_pandas.iloc[0]['o']
                    qty_pe = int(round((PORTFOLIO_ * INDEX_LEVERAGE_ / pe_strike) / LOT_SIZE_) * LOT_SIZE_)
                else:
                    entry_price_pe = float('nan')
                    qty_pe = 0
    
                print(f'CE Entry Price: {entry_price_ce}, PE Entry Price: {entry_price_pe}')
    
                tradebook = await send_trade(
                    strike=ce_strike, 
                    opt_type='C', 
                    expiry=expiry,
                    dte=dte,
                    current_date=current_date, 
                    current_time=search_time, 
                    entry_price=entry_price_ce, 
                    qty=qty_ce, 
                    tradebook=tradebook,
                )
                tradebook = await send_trade(
                    strike=pe_strike, 
                    opt_type='P', 
                    expiry=expiry,
                    dte=dte,
                    current_date=current_date, 
                    current_time=search_time, 
                    entry_price=entry_price_pe, 
                    qty=qty_pe, 
                    tradebook=tradebook,
                )
    
                take_new_trade = False
                rebalance = False
                position_active = True
    
            if not take_new_trade and position_active:
                for i, row in spot_df.iterrows():
                    # print(spot_df.iloc[i])
                    if row['c'] >= ce_strike or row['c'] <= pe_strike:
                        # SL Hit, Rebalance !
                        print(f'Rebalancing @ {row["datetime"]}')
                        current_timestamp = row['datetime']
                        search_time = (row['datetime']).time()
                        take_new_trade = True
                        rebalance = True
                        expiry_found = False
                        
                        ce_df = await fetch_data(
                            index=index_,
                            expiry=expiry,
                            strike=ce_strike,
                            asset_class='C',
                            start_date=current_date,
                            start_time=search_time,
                            end_date=current_date,
                            end_time=dt.time(15, 30),
                        )
                        if ce_df is not None and not isinstance(ce_df, str):
                            ce_df = resample(ce_df, tf, offset)
                            ce_df_pandas = ce_df.to_pandas()
                            exit_price_ce = ce_df_pandas.iloc[0]['c']
    
                        pe_df = await fetch_data(
                            index=index_,
                            expiry=expiry,
                            strike=pe_strike,
                            asset_class='P',
                            start_date=current_date,
                            start_time=search_time,
                            end_date=current_date,
                            end_time=dt.time(15, 30),
                        )
                        if pe_df is not None and not isinstance(pe_df, str):
                            pe_df = resample(pe_df, tf, offset)
                            pe_df_pandas = pe_df.to_pandas()
                            exit_price_pe = pe_df_pandas.iloc[0]['c']
    
                        tradebook = await take_exit(
                            strike=ce_strike, 
                            opt_type='C', 
                            exit_date=current_date, 
                            exit_time=search_time, 
                            tradebook=tradebook, 
                            exit_price=exit_price_ce
                        )
    
                        tradebook = await take_exit(
                            strike=pe_strike, 
                            opt_type='P', 
                            exit_date=current_date, 
                            exit_time=search_time, 
                            tradebook=tradebook, 
                            exit_price=exit_price_pe
                        )
                        print(f'CE Exit {exit_price_ce} , PE Exit {exit_price_pe}')
                        position_active = False
                        current_timestamp = row['datetime']
                        search_time = (row['datetime']).time()
                        break
                        
                    elif (row['datetime'].date() == expiry) and (row['datetime'].time() == dt.time(15, 20)):
                        # Expiry Exit, No Rebalancing
    
                        ce_df = await fetch_data(
                            index=index_,
                            expiry=expiry,
                            strike=ce_strike,
                            asset_class='C',
                            start_date=current_date,
                            start_time=search_time,
                            end_date=current_date,
                            end_time=dt.time(15, 30),
                        )
                        if ce_df is not None and not isinstance(ce_df, str):
                            ce_df = resample(ce_df, tf, offset)
                            ce_df_pandas = ce_df.to_pandas()
                            exit_price_ce = ce_df_pandas.iloc[-1]['c']
    
                        pe_df = await fetch_data(
                            index=index_,
                            expiry=expiry,
                            strike=pe_strike,
                            asset_class='P',
                            start_date=current_date,
                            start_time=search_time,
                            end_date=current_date,
                            end_time=dt.time(15, 30),
                        )
                        if pe_df is not None and not isinstance(pe_df, str):
                            pe_df = resample(pe_df, tf, offset)
                            pe_df_pandas = pe_df.to_pandas()
                            exit_price_pe = pe_df_pandas.iloc[-1]['c']
    
                        tradebook = await take_exit(
                            strike=ce_strike, 
                            opt_type='C', 
                            exit_date=current_date, 
                            exit_time=dt.time(15, 20), 
                            tradebook=tradebook, 
                            exit_price=exit_price_ce
                        )
    
                        tradebook = await take_exit(
                            strike=pe_strike, 
                            opt_type='P', 
                            exit_date=current_date, 
                            exit_time=dt.time(15, 20), 
                            tradebook=tradebook, 
                            exit_price=exit_price_pe
                        )
                        expiry_found = False
                        rebalance = False
                        take_new_trade = False
                        position_active = False
                        first_trade_of_month = False
    
                if not rebalance:
                    current_date += dt.timedelta(days=1)
                    search_time = dt.time(10, 0)
                if rebalance:
                    current_timestamp += dt.timedelta(minutes=1)
                    search_time = current_timestamp.time()

        else:
            current_date += dt.timedelta(days=1)
            continue

    tb =  pd.DataFrame(tradebook)
    if len(tb)> 0:
        tb['points'] = tb['entry price'] - tb['exit price']
        tb['slippage'] = (tb['entry price'] + tb['exit price']) * 0.01
        tb['final points'] = tb['points'] - tb['slippage']
        tb['PnL'] = tb['qty'] * tb['final points']
        tb['portfolio'] = PORTFOLIO_
        tb['index leverage'] = INDEX_LEVERAGE_
        tb['ROI%'] = tb['PnL'] * 100 / tb['portfolio']
    return tb

In [46]:
tb = await execute(bnf_1min, tf_, offset_)

2019-01-01 2019-01-01 10:00:00
Nearest Expiry : 2019-01-31, Spot ATM : 10822.45, DTE : 30
CE Strike : 11150, PE Strike : 10500
CE Entry Price: 60.0, PE Entry Price: 79.5
2019-01-02 2019-01-01 10:00:00
2019-01-03 2019-01-01 10:00:00
2019-01-04 2019-01-01 10:00:00
2019-01-07 2019-01-01 10:00:00
2019-01-08 2019-01-01 10:00:00
2019-01-09 2019-01-01 10:00:00
2019-01-10 2019-01-01 10:00:00
2019-01-11 2019-01-01 10:00:00
2019-01-14 2019-01-01 10:00:00
2019-01-15 2019-01-01 10:00:00
2019-01-16 2019-01-01 10:00:00
2019-01-17 2019-01-01 10:00:00
2019-01-18 2019-01-01 10:00:00
2019-01-21 2019-01-01 10:00:00
2019-01-22 2019-01-01 10:00:00
2019-01-23 2019-01-01 10:00:00
2019-01-24 2019-01-01 10:00:00
2019-01-25 2019-01-01 10:00:00
2019-01-28 2019-01-01 10:00:00
2019-01-29 2019-01-01 10:00:00
2019-01-30 2019-01-01 10:00:00
2019-01-31 2019-01-01 10:00:00
2019-02-01 2019-02-01 10:00:00
Nearest Expiry : 2019-02-28, Spot ATM : 10883.0, DTE : 27
CE Strike : 11200, PE Strike : 10600
CE Entry Price: 54.85,

In [47]:
tb.tail(500)

,entry date,entry time,index,strike,expiry,dte,option type,entry price,qty,exit date,exit time,exit price,points,slippage,final points,PnL,portfolio,index leverage,ROI%
0,2019-01-01,10:00:00,nifty,11150,2019-01-31,30,C,60.0000,7200,2019-01-31,15:20:00,0.0500,59.9500,0.6005,59.3495,427316.4000,10000000,8,4.2732
1,2019-01-01,10:00:00,nifty,10500,2019-01-31,30,P,79.5000,7650,2019-01-31,15:20:00,0.0500,79.4500,0.7955,78.6545,601706.9250,10000000,8,6.0171
2,2019-02-01,10:00:00,nifty,11200,2019-02-28,27,C,54.8500,7125,2019-02-19,15:12:00,4.0000,50.8500,0.5885,50.2615,358113.1875,10000000,8,3.5811
3,2019-02-01,10:00:00,nifty,10600,2019-02-28,27,P,85.5000,7575,2019-02-19,15:12:00,109.6000,-24.1000,1.9510,-26.0510,-197336.3250,10000000,8,-1.9734
4,2019-03-01,15:13:00,nifty,11150,2019-03-28,27,C,59.5500,7200,2019-03-11,13:14:00,141.7000,-82.1500,2.0125,-84.1625,-605970.0000,10000000,8,-6.0597
5,2019-03-01,15:13:00,nifty,10550,2019-03-28,27,P,70.5000,7575,2019-03-11,13:14:00,14.2000,56.3000,0.8470,55.4530,420056.4750,10000000,8,4.2006
6,2019-03-11,13:15:00,nifty,11350,2019-03-28,17,C,47.0000,7050,2019-03-13,15:24:00,126.8500,-79.8500,1.7385,-81.5885,-575198.9250,10000000,8,-5.7520
7,2019-03-11,13:15:00,nifty,10950,2019-03-28,17,P,53.0500,7275,2019-03-13,15:24:00,26.1000,26.9500,0.7915,26.1585,190303.0875,10000000,8,1.9030
8,2019-03-13,15:25:00,nifty,11500,2019-03-28,15,C,58.7500,6975,2019-03-18,09:15:00,124.6000,-65.8500,1.8335,-67.6835,-472092.4125,10000000,8,-4.7209
9,2019-03-13,15:25:00,nifty,11200,2019-03-28,15,P,60.0000,7125,2019-03-18,09:15:00,24.3000,35.7000,0.8430,34.8570,248356.1250,10000000,8,2.4836


In [48]:
tb['ROI%'].sum()

-45.179826750000004

In [49]:
tb.to_csv('condors.csv', index=False)

In [50]:
tb['DATETIME'] = tb['entry date'].astype(str) + ' ' + tb['entry time'].astype(str)
tb['DATETIME'] = pd.to_datetime(tb['DATETIME'])
tb['Trade Year'] = tb['DATETIME'].dt.year

In [51]:
tb['expiry'] = pd.to_datetime(tb['expiry'])
tb['entry date'] = pd.to_datetime(tb['entry date'])
tb['dte'] = (tb['expiry'] - tb['entry date']).dt.days

In [52]:
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,34.9187,42,57.1429,4.2597,-4.2071,-12.4024,2.8155,...
2020,-119.7539,164,48.7805,6.2286,-7.7255,-152.3687,-0.7859,...
2021,24.5227,60,56.6667,4.5011,-4.9429,-16.8401,1.4562,...
2022,-12.2182,78,51.2821,4.7378,-5.3087,-35.2610,-0.3465,...
2023,14.9856,36,69.4444,2.4209,-5.0596,-17.3919,0.8616,...
2024,-1.2607,52,51.9231,3.8957,-4.6280,-33.9753,-0.0371,...
2025,13.6259,24,54.1667,3.9077,-4.1305,-9.7475,1.3979,...
Overall,-45.1798,456,53.2895,4.7719,-5.9938,-152.3687,-0.2965,...
